# 🚀 BDA Contest - SOTA Solution with AutoGluon

## 핵심 전략: AutoGluon (AutoML)
- **최신 논문 기술 집약체**: Neural Network + XGBoost + LightGBM + CatBoost 자동 앙상블
- **Deep Learning**: 정형 데이터 전용 신경망(Tabular Neural Network) 자동 적용
- **NLP 통합**: 텍스트 컬럼(지원동기 등)을 NLP 모델로 임베딩하여 활용
- **Stacking & Bagging**: 수십 개 모델을 겹쳐 쌓아 과적합 방지 및 성능 극대화
- **Threshold Calibration**: 데이터 불균형 해결을 위한 분위수(Quantile) 기반 Cut-off

In [ ]:
# 1. AutoGluon 설치 (필수)
# 주의: 설치 후 런타임 재시작이 필요할 수 있습니다. (RESTART SESSION 버튼 클릭)
!pip install autogluon -q

import pandas as pd
import numpy as np
import re
import shutil
import os
from autogluon.tabular import TabularPredictor

print("Setup Complete! If you see errors, please Restart Session.")

In [ ]:
# 2. 데이터 로드
RAW_TRAIN_URL = 'https://raw.githubusercontent.com/choicompany/bdadacon/refs/heads/main/rawdata/train.csv'
RAW_TEST_URL = 'https://raw.githubusercontent.com/choicompany/bdadacon/refs/heads/main/rawdata/test.csv'

train = pd.read_csv(RAW_TRAIN_URL)
test = pd.read_csv(RAW_TEST_URL)

print(f"Train Shape: {train.shape}, Test Shape: {test.shape}")

In [ ]:
# 3. 최소한의 피처 엔지니어링

def count_items(text):
    if pd.isna(text) or str(text).strip() == '': return 0
    return str(text).count(',') + 1

multi_select_cols = [
    'certificate_acquisition', 'desired_certificate', 
    'desired_job', 'desired_job_except_data',
    'onedayclass_topic', 'expected_domain'
]

for df in [train, test]:
    for col in multi_select_cols:
        if col in df.columns:
            df[f'{col}_count'] = df[col].apply(count_items)
            
    if 'major_data' in df.columns:
        df['major_data'] = df['major_data'].astype(str).str.lower().map({'true': 1, 'false': 0}).fillna(0).astype(int)

print("Feature Engineering Done.")

In [ ]:
# 4. AutoGluon 학습 (Highest Quality)
# [수정됨] 기존 모델 폴더 있으면 삭제 (Learner is already fit 오류 방지)

metric = 'f1'
label = 'completed'
save_path = 'ag_models_best'

if os.path.exists(save_path):
    shutil.rmtree(save_path)
    print(f"Deleted existing model folder: {save_path}")

predictor = TabularPredictor(
    label=label,
    eval_metric=metric,
    path=save_path
).fit(
    train,
    presets='best_quality',
    time_limit=60*5,         # 5분 학습 요청 반영
    ag_args_fit={'num_gpus': 1}
)

In [ ]:
# 5. 성능 평가
predictor.leaderboard(train, silent=True)

In [ ]:
# 6. 예측 및 제출 (Threshold 보정)
test_data = test.drop(columns=['ID'])
pred_probs = predictor.predict_proba(test_data)
prob_1 = pred_probs.iloc[:, 1]

print("확률통계:", prob_1.describe())

# 상위 30%만 1로 설정
target_ratio = 0.30
threshold = prob_1.quantile(1 - target_ratio)

print(f"\nThreshold: {threshold:.4f}")

final_preds = (prob_1 >= threshold).astype(int)
print(f"결과 분포: 0={sum(final_preds==0)}, 1={sum(final_preds==1)}")

submission = pd.DataFrame({
    'ID': test['ID'],
    'completed': final_preds
})

submission.to_csv('submission.csv', index=False)
print("Saved: submission.csv")
print(submission.head())

In [ ]:
# 7. 다운로드
from google.colab import files
try:
    files.download('submission.csv')
except:
    pass